In [1]:
import requests as r
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import scrapy
import json
import re

In [6]:
df = pd.read_excel('data.xlsx', sheet_name='data')

In [339]:
df.head(3)

,file_name,adress,oktmo,type_house,condition,s_total,s_residental,RF_own,RF_LOCAL_own,type_community,demolition_date,id
0,Сведения по ОЖФ Москва г на 09-07-2023_1.csv,"101000, Москва г, б-р. Покровский, д. 4/17, ст...",45375000,Многоквартирный,Исправный,5149.2,2745.0,-,-,-,NaN,https://dom.gosuslugi.ru/homemanagement/api/re...
1,Сведения по ОЖФ Москва г на 09-07-2023_1.csv,"101000, Москва г, б-р. Покровский, д. 4/17, ст...",45375000,Многоквартирный,Исправный,995.2,839.5,-,-,-,NaN,https://dom.gosuslugi.ru/homemanagement/api/re...
2,Сведения по ОЖФ Москва г на 09-07-2023_1.csv,"101000, Москва г, б-р. Покровский, д. 4/17, ст...",45375000,Многоквартирный,Исправный,1465.8,1099.2,-,-,-,NaN,https://dom.gosuslugi.ru/homemanagement/api/re...


In [121]:
# Материал дома
# response['intWallMaterialList']

# Износ
# response['deterioration']

# Год постройки
# response['buildingYear']

# Год ввода в эксплуатацию
# operationYear

# Серия дома
# response['planSeries']

# Дата определения износа
# response['deteriorationDate']

# Стадия жизненного цикла:
# response['lifeCycleStage']

In [122]:
# start moscow_houses_spider in terminal
# get result.json

In [2]:
df_pars = pd.read_json('result.json')

In [3]:
df_pars['intWallMaterialList'] = [x['intWallMaterialList'] for x in df_pars.result]
df_pars['deterioration'] = [x['deterioration'] for x in df_pars.result]
df_pars['buildingYear'] = [x['buildingYear'] for x in df_pars.result]
df_pars['operationYear'] = [x['operationYear'] for x in df_pars.result]
df_pars['planSeries'] = [x['planSeries'] for x in df_pars.result]
df_pars['deteriorationDate'] = [x['deteriorationDate'] for x in df_pars.result]
df_pars['lifeCycleStage'] = [x['lifeCycleStage']['lifeCycleStage'] 
                               if x['lifeCycleStage'] != None 
                               else x['lifeCycleStage'] for x in df_pars.result]

df_pars['houseCondition'] = [x['houseCondition']['houseCondition'] 
                               if x['houseCondition'] != None 
                               else x['houseCondition'] for x in df_pars.result]
df_pars = df_pars.drop(['result'], axis=1)
df_pars.rename(columns={'url':'id'}, inplace=True)

In [8]:
df_result = df.merge(df_pars, left_on='id', right_on='id', how='left')

In [9]:
# Корректируем столбцы
# Заполняем пустые ячейки значением 'Нет данных'
df_result.lifeCycleStage.replace(to_replace=[None], value='Нет данных', inplace=True)
df_result.houseCondition.replace(to_replace=[None], value='Нет данных', inplace=True)
df_result.intWallMaterialList.replace(to_replace=[None, ''], value='Нет данных', inplace=True)

# Итоговый год с датой строительства/начала эксплуатации дома
df_result['correctYear'] = [x[0:4] if x != None else y for x,y in zip(df_result.buildingYear, df_result.operationYear)]
df_result = df_result.drop(['buildingYear', 'operationYear'], axis=1)

# Укрупняем столбец с информацией о материале дома
df_result.intWallMaterialList = [ x.replace(';', ',') for x in df_result.intWallMaterialList ]
pattern = r'[A-Я][аА-яЯ| |\-|\/]*[^,]'
df_result.intWallMaterialList = [ re.findall(pattern, x)[0] for x in df_result.intWallMaterialList ]

# Правим столбец с данным о текущем износе
df_result.deterioration = [ x.replace(',', '.') if x != None else x for x in df_result.deterioration ]
df_result.deterioration = df_result.deterioration.astype(float)
df_result.deterioration = [ x / 100 for x in df_result.deterioration]

df_result.correctYear = df_result.correctYear.fillna(0)
df_result.correctYear.replace(to_replace=['12'], value=0, inplace=True)
df_result.correctYear = df_result.correctYear.astype(int)

In [10]:
df_result = df_result.drop(['condition', 'RF_own', 'RF_LOCAL_own', 'type_community', 'demolition_date', 'id'],  axis=1)

In [11]:
df_result.to_excel('full_data.xlsx')